---
author: Fenke Meijer

format:
  html:
    mathjax: true
    embed-resources: true
  typst:
    output-file: barn_mass_balance.pdf
---

# Barn Mass Balance
> Mass balance calculations for a barn following Wageningen UR model.

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| default_exp modules.barn_mass_balance

In [ ]:
#| export
import inspect
import numpy as np
import pandas as pd
from scipy.constants import gas_constant

from pandas import DataFrame
from corebridge.aicorebridge import AICoreModule



In [ ]:
import os, json

In [ ]:
#| exporti
kelvin_zero = -273.15

In [ ]:
#| exporti
MOLECULAR_MASS = dict(
    co2=44.01,
    nh3=17.031,
    ch4=16.043
)

MOLECULAR_NAMES = { 
    k: ([v] if isinstance(v, str) else list(v)) + [k]
    for k, v in dict(
        co2=['carbon_dioxide', 'carbon dioxide', 'kooldioxide', 'co2'],
        nh3=['ammonia', 'nh3', 'ammoniak'],
        ch4=['methane', 'methaan', 'ch4']
    ).items() 
}


## Achtergrond

In Nederland is men momenteel vooral geïnteresseerd in de emissie uit stallen
met dieren. We kunnen onderscheid maken tussen 2 type stallen: de natuurlijk 
geventileerde stallen (open stallen) en de mechanisch geventileerde stallen 
(gesloten stallen). Over het algemeen zitten melkkoeien (en -geiten) in open 
stallen en zitten de intensievere dieren (pluimvee, varkens, kalveren) in 
gesloten stallen.

### Emissieberekening 

Emissie wordt berekend door de gemeten concentratie van een bepaalde stof 
te vermenigvuldigen met het ventilatiedebiet. De eenheid van emissie wordt
uitgedrukt in massa/tijdseenheid” (zoals kg/uur). 

### Concentratiemeting

De concentratie van de stof wordt gemeten met sensoren. Deze worden op 
bepaalde plekken bij de luchtinlaat en luchtuitlaat van de stal gehangen. 
Er is een consensus dat de gemeten concentraties dan betrouwbaar en 
representatief zijn.

### Ventilatiedebiet

#### Mechanisch geventileerde stallen

In mechanisch geventileerde stallen wordt het ventilatiedebiet bepaald met behulp van
meetwaaiers. Deze meetwaaiers worden op ventilatoren geplaatst en kunnen meten hoeveel lucht
er per tijdseenheid door de ventilator wordt geblazen. Omdat mechanisch geventileerde stallen
maar één (of enkele) uitstroomopeningen hebben, kan op deze manier worden bepaald wat het
totale ventilatiedebiet van een stal is. Er is een consensus dat het bepalen van ventilatiedebiet met
behulp van meetwaaiers betrouwbaar en representatief is.

#### Natuurlijk geventileerde stallen

Natuurlijk geventileerde stallen zijn voornamelijk open, waardoor het niet mogelijk is om met
meetwaaiers te bepalen wat het debiet van de stallen is. Daarom wordt bij deze stallen gebruik
gemaakt van de CO~2~ massabalans. Dit is een theoretische benadering gebaseerd op verschillende
parameters die door de tijd heen kunnen veranderen.

### Emissieberekening met CO~2~ massabalans 

De ratiomethode is een veelgebruikte methode om de emissie van ammoniak (NH~3~) in 
stallen te schatten. Deze methode maakt gebruik van de concentratie van CO~2~ als 
tracergas, omdat CO~2~ een relatief constante productie heeft in de stal en goed te 
berekenen en meten is. De basis van de ratiomethode is het idee dat de verhouding 
tussen de concentraties van CO~2~  binnen en buiten de stal een indicatie geeft van 
het ventilatiedebiet en daarmee de totale emissie van NH~3~.

Een randvoorwaarde van de ratiomethode is dat de concentraties van NH~3~ en het 
tracergas - in dit geval CO~2~  - op dezelfde meetpunten en met dezelfde meetfrequentie
gemeten moeten worden. Om een goede schatting van de emissie te verkrijgen is het 
van belang dat *de concentratieratio’s per meetpunt worden geschat en daarna
een gemiddeld van deze waarden wordt genomen*, in plaats dan eerst een gemiddelde 
concentratie van al die punten te bepalen en daarna de ratiomethode te gebruiken.


## Support calculations

### Gas density

The density of a gas can be calculated using the ideal gas law:
$$
\rho = \frac{M p}{R \cdot T}
$$

where:
* $M$ is the molar mass of the gas in kg/mol

* $p$ is the pressure in Pascals

* $T$ is the temperature in Kelvin

* $R$ is the universal gas constant


In [ ]:
#| export

def gas_density(
        P:float,        # pressure in Pascal
        T:float,        # temperature in Kelvin
        ppm:float,      # measured parts per million
        molweight:float # molecular weight in grams per mole
    ):
    '''Calculates mass density in grams per cubic metre
    P : pressure in Pa
    T : temperature in degrees Kelvin
    ppm : measured parts per million
    molweight: molecular weight in grams per mole'''

    return (ppm/1000000) * molweight * P / (gas_constant * T)


### Gasconcentraties in de lucht

Concentraties van chemicaliën in de lucht worden meestal gemeten als de massa van chemicaliën (milligram, microgram, nanogram of picogram) per volume lucht (kubieke meter of kubieke voet). Concentraties kunnen ook worden uitgedrukt als delen per miljoen (ppm) of delen per miljard (ppb) door gebruik te maken van een conversiefactor. Deze conversiefactor is gebaseerd op het moleculair gewicht van de chemische stof en is voor elke chemische stof verschillend. Typisch worden conversies voor chemicaliën in de lucht gemaakt met een veronderstelling van een druk van 1 atmosfeer en een temperatuur van 25 graden Celsius. Voor deze omstandigheden is de vergelijking om te converteren van concentratie in delen per miljoen naar concentratie in milligram per kubieke meter (mg/m3) als volgt:

$$	\text{Concentratie (mg/m3)} = 0.0409 	\times 	\text{concentratie (ppm)} 	\times 	\text{moleculair gewicht}$$

#### Concentrations of chemicals

Concentrations of chemicals in the air are usually measured as the mass of chemicals (milligrams, micrograms, nanograms or picograms) per volume of air (cubic meters or cubic feet). Concentrations can also be expressed as parts per million (ppm) or parts per billion (ppb) by using a conversion factor. This conversion factor is based on the molecular weight of the chemical and it is different for every chemical. The temperature of the atmosphere also has an influence on the calculation.

Typically, conversions for chemicals in air are made as
suming a pressure of 1 atmosphere and a temperature of 
25 degrees Celsius. For these conditions, the equation to 
convert from concentration in parts per million to con
centration in milligrams per cubic meter (mg/m3) is as 
follows:
$$
\text{Concentration (mg/m3)} = 0.0409 \times \text{concentration (ppm)} \times \text{molecular weight}
$$



#### Functie implementatie

In [ ]:
#| export

def gas_density_from_sensor_measurment(
        ppm:float,          # measured parts per million
        molweight:float):   # molecular weight in grams per mole
    '''Calculates mass density in milligrams per cubic metre'''

    return round(0.0409 * ppm * molweight,5)


### CO~2~ productie

De CO~2~ productie in een stal (in m^3^ / uur) kan worden berekend met behulp van de volgende formules voor melkvee en pinken

#### Melkvee

$$
P CO_2 = 0.2 \frac{5.6 m^{0.75} + 22 Y_1 + 1.6 \times 10^{-5} p^3}{1000}
$$

Where:

* $m$ is the live weight in kg
* $Y_1$ is the daily milk production in kg per dier per dag
* $p$ number of dracht dagen


##### Functie implementatie



In [ ]:
#| export

# =(5.6*(B10^0.75)+22*B7+1.6*(10^-5)*(B14^3))*B3/1000

def PCO2_melkvee(
        aantal,             # number of animals
        melkproductie,      # milk production in kg per animal per day
        drachtdagen,        # days carrying (average)
        gewicht             # average weight of the animals in kg 
    ):
    '''CO2 productie van melkvee per dier per dag
    gewicht: (gemiddelde) gewicht van de dieren
    melkproductie: melkproductie in kg per dier per dag
    drachtdagen: gemiddelde drachttijd (in dagen)
    De defaults zijn voor droogstaande koeien'''
    return aantal * 0.2 * (
        5.6 * np.pow(gewicht, 0.75) +
        22 * melkproductie +
        1.6e-5 * np.pow(drachtdagen, 3.0)
    ) / 1000.0


In [ ]:
test_args_melkvee = dict(
    aantal=130,
    melkproductie=28,
    drachtdagen=160,
    gewicht=650
)

PCO2_melkvee(**test_args_melkvee) 

np.float64(36.46325050213528)

In [ ]:
test_args_droog = dict(
    aantal=6,
    melkproductie=0,
    drachtdagen=220,
    gewicht=650
)

PCO2_melkvee(**test_args_droog)

np.float64(1.0695176539447053)

#### Pinken

$$
P CO_2 = 0.2 \frac{7.64 m^{0.69} + Y_2 (\frac{23}{M} - 1) (\frac{57.27 + 0.302 m}{1 - 0.171 Y_2})  + 1.6 \times 10^{-5} p^3}{1000}
$$

Where:

* $m$ is the live weight in kg

* $M$ is the energy content of their food in MJ per kg

* $Y_2$ is the daily weight gain in kg per dier per dag

* $p$ number of dracht dagen

##### Functie implementatie



In [ ]:
#| export

def PCO2_pinken(
        aantal,             # number of animals
        energievoeding,     # energy feed
        drachtdagen,        # days carrying (average)
        gewicht,            # average weight of the animals in kg 
        gewichtstoename     # average weight gain of the animals in kg per day
    ):
        
    '''CO2 productie van pinken'''
    
    return aantal * 0.2 * (
        7.64 * np.pow(gewicht, 0.69) +
        gewichtstoename * 
            (23.0 / energievoeding - 1.0) * (
                (57.27 + 0.302 * gewicht) / 
                (1 - 0.171 * gewichtstoename) 
            ) +
        1.6e-5 * np.pow(drachtdagen, 3.0)
    ) / 1000.0




In [ ]:
test_args_pinken = dict(
    aantal=0,
    energievoeding=10,
    drachtdagen=140,
    gewicht=400,
    gewichtstoename=0.6
)
PCO2_pinken(**test_args_pinken)

np.float64(0.0)

In [ ]:
test_args_pinken_niet_drachtig = dict(
    aantal=0,
    energievoeding=10,
    drachtdagen=0,
    gewicht=250,
    gewichtstoename=0.6
)
PCO2_pinken(**test_args_pinken_niet_drachtig)

np.float64(0.0)

#### Temperatuur correctie

Temperatuur heeft invloed op spijsveetering en gedrag en daarmee op de CO~2~ productie, correctie kan worden toegepast met de volgende formule:

$$
P CO_2 (T) = P CO_2 \times (1000  + 4 \times (20 - T_{stal})) / 1000
$$

In [ ]:
#| export

def calculate_temperatuur_correctie(temperatuur):
    '''Calculate temperature correction factor for CO2 production'''
    return (1000.0 + 4.0 * (20.0 - temperatuur)) / 1000.0

def PCO2_temperatuurcorrectie(
        pco2,           # calculated CO2 production in cubic meters per hour
        temperatuur     # temperature in the barn in degrees Celsius
    ):
    '''Bereken temperatuur correctie voor de CO2 productie'''
    
    return pco2 * calculate_temperatuur_correctie(temperatuur)


#### PcO~2~ functie categorie mapping

In [ ]:
dict(inspect.signature(PCO2_melkvee).parameters)

{'aantal': <Parameter "aantal">,
 'melkproductie': <Parameter "melkproductie">,
 'drachtdagen': <Parameter "drachtdagen">,
 'gewicht': <Parameter "gewicht">}

In [ ]:
#| export

pco2_category_functions_mapping = {
    "melkvee": PCO2_melkvee,
    "droogstaande koeien": PCO2_melkvee,
    "drachtig jongvee": PCO2_pinken,
    "niet drachtig jongvee": PCO2_pinken,
}
pco2_category_functions_parameters = {
    k: dict(inspect.signature(v).parameters)
    for k, v in pco2_category_functions_mapping.items()
}

In [ ]:
pco2_category_functions_parameters

{'melkvee': {'aantal': <Parameter "aantal">,
  'melkproductie': <Parameter "melkproductie">,
  'drachtdagen': <Parameter "drachtdagen">,
  'gewicht': <Parameter "gewicht">},
 'droogstaande koeien': {'aantal': <Parameter "aantal">,
  'melkproductie': <Parameter "melkproductie">,
  'drachtdagen': <Parameter "drachtdagen">,
  'gewicht': <Parameter "gewicht">},
 'drachtig jongvee': {'aantal': <Parameter "aantal">,
  'energievoeding': <Parameter "energievoeding">,
  'drachtdagen': <Parameter "drachtdagen">,
  'gewicht': <Parameter "gewicht">,
  'gewichtstoename': <Parameter "gewichtstoename">},
 'niet drachtig jongvee': {'aantal': <Parameter "aantal">,
  'energievoeding': <Parameter "energievoeding">,
  'drachtdagen': <Parameter "drachtdagen">,
  'gewicht': <Parameter "gewicht">,
  'gewichtstoename': <Parameter "gewichtstoename">}}

#### PCO~2~ Parameters

Voor het CO2-productiemodel zijn een aantal productiegegevens nodig. Melkproductie en 
–samenstelling worden altijd gemeten en gerapporteerd. De andere benodigde parameters 
(diergewicht, dagen in dracht, en voor jongvee de energiewaarde van het voer en 
gewichtstoename), worden bij voorkeur op basis van metingen op de bedrijfslocaties 
vastgesteld. Wanneer deze niet beschikbaar zijn dienen de volgende standaardwaarden 
voor te worden gebruikt. 

In [ ]:
#| export
#| hide

_default_parameters = [
    dict(
        categorie='melkvee',
        gewicht=650,  # kg
        drachtdagen=160,  # days
    ),
    dict(
        categorie='droogstaande koeien',
        gewicht=650,  # kg
        drachtdagen=220,  # days
        melkproductie=0,  # kg per day
    ),
    dict(
        categorie='drachtig jongvee',
        gewicht=400,  # kg
        drachtdagen=140,  # days
        energievoeding=10.0,  # MJ NEL per kg
        gewichtstoename=0.6,  # kg per day
    ),
    dict(
        categorie='niet drachtig jongvee',
        gewicht=250,  # kg
        drachtdagen=0,  # days
        energievoeding=10.0,  # MJ NEL per kg
        gewichtstoename=0.6,  # kg per day
    ),
]

In [ ]:
pd.DataFrame(_default_parameters).set_index('categorie')

,gewicht,drachtdagen,melkproductie,energievoeding,gewichtstoename
categorie,,,,,
melkvee,650,160,NaN,NaN,NaN
droogstaande koeien,650,220,0.0,NaN,NaN
drachtig jongvee,400,140,NaN,10.0,0.6
niet drachtig jongvee,250,0,NaN,10.0,0.6


In [ ]:
#| export
default_pco2_parameters = {
    kwargs_item.pop('categorie'): kwargs_item.copy()
    for kwargs_item in _default_parameters
}


In [ ]:
print(json.dumps(default_pco2_parameters, indent=4))

{
    "melkvee": {
        "gewicht": 650,
        "drachtdagen": 160
    },
    "droogstaande koeien": {
        "gewicht": 650,
        "drachtdagen": 220,
        "melkproductie": 0
    },
    "drachtig jongvee": {
        "gewicht": 400,
        "drachtdagen": 140,
        "energievoeding": 10.0,
        "gewichtstoename": 0.6
    },
    "niet drachtig jongvee": {
        "gewicht": 250,
        "drachtdagen": 0,
        "energievoeding": 10.0,
        "gewichtstoename": 0.6
    }
}


In [ ]:
#| export

def create_pco2_function_mapping_from_parameters(pco2_parameters):
    '''Create a mapping of category to PCO2 calculation functions'''

    parameters = default_pco2_parameters.copy()
    parameters.update(pco2_parameters)
    #print(json.dumps(parameters, indent=4))

    return {
        'melkvee': lambda aantal, **kwargs: PCO2_melkvee(
            aantal=aantal,
            melkproductie=kwargs.get('melkproductie', parameters['melkvee']['melkproductie']),  # kg per day
            drachtdagen=kwargs.get('drachtdagen', parameters['melkvee']['drachtdagen']),  # days
            gewicht=kwargs.get('gewicht',  parameters['melkvee']['gewicht'])  # kg
        ),
        'droogstaande koeien': lambda aantal, **kwargs: PCO2_melkvee(
            aantal=aantal,
            melkproductie=kwargs.get('melkproductie', 0),  # kg per day
            drachtdagen=kwargs.get('drachtdagen', parameters['droogstaande koeien']['drachtdagen']),  # days
            gewicht=kwargs.get('gewicht', parameters['droogstaande koeien']['gewicht'])  # kg
        ),
        'drachtig jongvee': lambda aantal, **kwargs: PCO2_pinken(
            aantal=aantal,
            energievoeding=kwargs.get('energievoeding', parameters['drachtig jongvee']['energievoeding']),  # MJ NEL per kg
            drachtdagen=kwargs.get('drachtdagen', parameters['drachtig jongvee']['drachtdagen']),  # days
            gewicht=kwargs.get('gewicht', parameters['drachtig jongvee']['gewicht']),  # kg
            gewichtstoename=kwargs.get('gewichtstoename', parameters['drachtig jongvee']['gewichtstoename'])  # kg per day
        ),
        'niet drachtig jongvee': lambda aantal, **kwargs: PCO2_pinken(
            aantal=aantal,
            energievoeding=kwargs.get('energievoeding', parameters['niet drachtig jongvee']['energievoeding']),  # MJ NEL per kg
            drachtdagen=kwargs.get('drachtdagen', parameters['niet drachtig jongvee']['drachtdagen']),  # days
            gewicht=kwargs.get('gewicht', parameters['niet drachtig jongvee']['gewicht']),  # kg
            gewichtstoename=kwargs.get('gewichtstoename', parameters['niet drachtig jongvee']['gewichtstoename'])  # kg per day
        )
    }

In [ ]:
#| export

def PCO2_calculation_from_mapping(
        mapping,
        category,
        aantal,
        **kwargs
):
    return mapping.get(category)(
        aantal=aantal,
        **kwargs
)

### Test berekeningen

### Emissie berekeningen

#### Ratiomethode

De ammoniakemissies (E~i~; in kg/jaar per dierplaats) worden per meetdag $i$ bepaald 
op basis van de geschatte CO~2~ -
productie in de stal (PCO~2i~; in m^3^ CO~2~ /uur), en de gemiddelde concentratieratio van CO~2~  en NH~3~ als CR~i~ over alle
meetpunten m waar CO~2~ - en NH~3~ concentraties tegelijkertijd in de stal gemeten zijn:



$$
E_i = PCO_{2i} \cdot CR_i 
$$

Voor $CR_i$  

$$
CR_i = \frac{1}{m} \sum_{m} \frac{(NH_{3})_{im}^{stal} - (\overline{NH_{3}})_i^{buiten} }{(CO_{2})_{im}^{stal} - (\overline{CO_{2}})_i^{buiten}  }
$$

$$
\overline{X_{i}^{buiten}} = \sum_{m} X_{i}^{buiten}
$$


Waarin

$X_{im}^{stal}$ 
: het 24-uurs gemiddelde van de concentratie van stof $X$ in stal op meetdag $i$ en op meetpunt $m$ 

$X_{im}^{buiten}$ 
: het 24-uurs gemiddelde van de concentratie van stof $X$ in de ingaande lucht op meetdag $i$ en op meetpunt $m$


$\overline{X_{i}^{buiten}}$
: het 24-uurs gemiddelde van de concentratie van stof $X$ in de ingaande lucht op meetdag $i$ over alle meetpunten






$$
\overline{(NH_{3})}_{i}^{buiten} = \frac{1}{m} \sum_{m} (NH_{3})_{i}^{buiten}
$$

and 
$$
\overline{CO_{2i}}^{buiten} = \frac{1}{n} \sum_{k=1}^{n} (CO_{2})_{ik}^{buiten}
$$

#### Ratiomethode met twee meetpunten

Wanneer er slechts twee meetpunten zijn, een binnen en een buiten, dan vervalt de berekening van de gemiddelden over meetpunten en kan de emissie worden berekend met de vereenvoudiging van $CR_i$:


$$
CR_i = \frac{(NH_{3})_{i}^{stal} - (NH_{3})_{i}^{buiten} }{(CO_{2})_{i}^{stal} - CO_{2i}^{buiten} }
$$

en 

$$
E_i = PCO_{2i} \cdot CR_i
$$

wordt

$$
E_i = PCO_{2i} \cdot \frac{(NH_{3})_{i}^{stal} - (NH_{3})_{i}^{buiten} }{(CO_{2})_{i}^{stal} - CO_{2i}^{buiten} }
$$


## Implementatie ratiomethode

We verwachten dat de gebruiker de volgende data als timeseries dataframe aanlevert:

| Kolomnaam       | Omschrijving                             | Eenheid           |
| :-------------- | :--------------------------------------- | :---------------- |
| CO2_stal        | CO2 concentratie in de stal in ppm       | ppm               |
| CO2_buiten      | CO2 concentratie buiten de stal in ppm   | ppm               |
| NH3_stal        | NH3 concentratie in de stal in ppm       | ppm               |
| NH3_buiten      | NH3 concentratie buiten de stal in ppm   | ppm               |
| temperatuur     | Temperatuur in de stal in Celcius        | °C                |

Daarnaast verwachten we dat de gebruiker de volgende gegevens meegeeft

bezetting
: aantal dieren in de stal per categorie als dictionary met categorie als key en aantal als value

parameters
: dictionary met de parameters voor de verschillende categorieën. Missende waardes voor parameters worden aangevult uit de volgende standaard parameters:




In [ ]:
pd.DataFrame(default_pco2_parameters).transpose()

,gewicht,drachtdagen,melkproductie,energievoeding,gewichtstoename
melkvee,650.0,160.0,NaN,NaN,NaN
droogstaande koeien,650.0,220.0,0.0,NaN,NaN
drachtig jongvee,400.0,140.0,NaN,10.0,0.6
niet drachtig jongvee,250.0,0.0,NaN,10.0,0.6


### Externe data

Data voor verificatie van de implementatie wordt veelal aangeleverd in excel werkboeken. Deze data kan worden ingelezen en aangepast aan onze behoeften.

#### VERA data

In [ ]:
#| exporti

def flatten_column_mapping(column_mapping: dict) -> list:
    '''Flatten the column mapping dictionary to a list of columns'''
    result = []
    for values in column_mapping.values():
        if isinstance(values, dict):
            result += flatten_column_mapping(values)
        elif isinstance(values, list):
            result += values
        else:
            result += [values]
    return result


In [ ]:
vera_data_filename = os.path.join(os.getcwd(), '..', 'data', 'massabalans', 'Rekenbestand emissie VERA.xlsx')
vera_dataframe = pd.read_excel(
    vera_data_filename, 
    sheet_name='Emissions (daily means)', 
    header=3, 
    index_col=7, 
    parse_dates=True
).drop([
    ' C1:                  cows >= 70%',
    'C2:                          Occupation rate >= 90%',
    'C3:                                                             milk production > 25',
    ' C1:                  heifers < 30%',
    'C2:                          Occupation rate >= 80%',
    'C3:                                                             milk production > 25.1',
    'C4:                                      urea content in milk > 15',
    'C5:                            dry cows < 25%'], axis=1
).dropna(axis=1, how='all')

In [ ]:
vera_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2011-04-04 to 2012-02-16
Data columns (total 55 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Measurement institute                                  60 non-null     object 
 1   Animal Category                                        60 non-null     object 
 2   Housing system                                         60 non-null     object 
 3   Measurement location                                   60 non-null     object 
 4   Measurement period                                     60 non-null     int64  
 5   Measurement day (in period)                            60 non-null     int64  
 6   Day in year                                            60 non-null     int64  
 7   Outside temperature [oC]                               24 non-null     float64
 8   Outside RH [%]                  

### Exctractie van werkbare data uit gegeven werkboeken


#### Warmte & CO~2~ data

In [ ]:
data = vera_dataframe.copy()
datacolumns = set(data.columns)


In [ ]:
#| export

def find_production_column_names(data: DataFrame):
    '''Find the column names for the co2 production columns in the VERA data'''

    datacolumns = set(data.columns)
    columnnames = {
        'drachtdagen': [col for col in datacolumns if 'drachtdagen' in col.lower() or 'pregnancy' in col.lower()],
    }

    datacolumns = set(data.columns) - set(flatten_column_mapping(columnnames))
    columnnames.update({
        'energievoeding': [col for col in datacolumns if 'energy' in col.lower() or 'energie' in col.lower()],
    })
    
    datacolumns = set(data.columns) - set(flatten_column_mapping(columnnames))
    columnnames.update({
            'melkproductie': [
                col 
                for col in datacolumns 
                if ('milk production' in col.lower() and 'C3' not in col ) or 'melkproductie' in col.lower()
            ],
    })
    
    datacolumns = set(data.columns) - set(flatten_column_mapping(columnnames))
    columnnames.update({
            'gewichtstoename': [
                col 
                for col in datacolumns 
                if 'weight gain' in col.lower() or 'gewichtstoename' in col.lower()
            ]
    })

    datacolumns = set(data.columns) - set(flatten_column_mapping(columnnames))
    columnnames.update({
            'gewicht': [col for col in datacolumns if 'weight' in col.lower() or 'gewicht' in col.lower()],
    })

    datacolumns = set(data.columns) - set(flatten_column_mapping(columnnames))
    columnnames.update({
            'remaining_columns': list(datacolumns),
    })

    return  columnnames

In [ ]:
print(json.dumps(find_production_column_names(vera_dataframe), indent=3))

{
   "drachtdagen": [
      "Days in pregnancy (heifers)",
      "Days in pregnancy (dry cows)",
      "Days in pregnancy (milking cows)"
   ],
   "energievoeding": [
      "Energy value of feed (heifers; MJ/kg dry matter)"
   ],
   "melkproductie": [
      "Milk production [kg/animal/day]"
   ],
   "gewichtstoename": [
      "Weight gain heifers [kg/day]"
   ],
   "gewicht": [
      "Weight heifers (pregnant) [kg]",
      "Weight heifers (not pregnant) [kg]",
      "Weight milking cows [kg]",
      "Weight dry cows [kg]"
   ],
   "remaining_columns": [
      "Dry cows vs. dairy cows (%)",
      "Closed cubicles",
      "NH3 outside [mg/m3]",
      "Summary.1",
      "Day in year",
      "% closed cubicles",
      "Housing system",
      "Animal Category",
      "CO2 outside [ppm]",
      "NH3 Emission [kg/year per animal place]",
      "Animal places",
      "NH3 inside [mg/m3]",
      "Ventilation rate [m3/h]",
      "CO2 inside [ppm]",
      "Dairy cows (milking + dry)",
      "Numb

In [ ]:
print(json.dumps(default_pco2_parameters, indent=4))

{
    "melkvee": {
        "gewicht": 650,
        "drachtdagen": 160
    },
    "droogstaande koeien": {
        "gewicht": 650,
        "drachtdagen": 220,
        "melkproductie": 0
    },
    "drachtig jongvee": {
        "gewicht": 400,
        "drachtdagen": 140,
        "energievoeding": 10.0,
        "gewichtstoename": 0.6
    },
    "niet drachtig jongvee": {
        "gewicht": 250,
        "drachtdagen": 0,
        "energievoeding": 10.0,
        "gewichtstoename": 0.6
    }
}


In [ ]:
#| export
def extract_production_column_names(
    data: DataFrame # DataFrame with measurement data
) -> dict:
    
    '''Extract column names for the co2 production columns from the DataFrame'''

    columnnames = find_production_column_names(data)  

    def is_heifer(colname):
        return 'heifer' in colname.lower() or 'jongvee' in colname.lower() or 'pinken' in colname.lower()
    def is_cow(colname):
        return 'cow' in colname.lower() or 'koeien' in colname.lower() or 'melkvee' in colname.lower()
    def is_dry(colname):
        return ('dry' in colname.lower() or 'droog' in colname.lower()) and 'milking' not in  colname.lower()
    def is_milking(colname):
        return ('milking' in colname.lower() or 'melk' in colname.lower() or 'melkvee' in colname.lower()) and 'dry' not in colname.lower()
    def is_not_pregnant(colname):
        return 'not pregnant' in colname.lower() or 'niet drachtig' in colname.lower() 
    def is_pregnant(colname):
        return ('pregnant' in colname.lower() or 'drachtig' in colname.lower() or 'pregnancy' in colname.lower()) and not is_not_pregnant(colname)

    categories = {
        'melkvee': {
            'gewicht' : [
                cn 
                for cn in columnnames['gewicht'] 
                if is_milking(cn) and not is_heifer(cn)
            ],
            'drachtdagen' : [
                cn 
                for cn in columnnames['drachtdagen'] 
                if is_pregnant(cn) and not is_heifer(cn) and not is_dry(cn)
            ],
            'melkproductie' : [
                cn 
                for cn in columnnames['melkproductie'] 
                
            ],
            'aantal' : [
                cn 
                for cn in columnnames['remaining_columns']
                if is_cow(cn) and is_milking(cn) and not is_heifer(cn) and not '%' in cn and not 'production' in cn
            ]
        },
        'droogstaande koeien': {
            'gewicht' : [
                cn 
                for cn in columnnames['gewicht'] 
                if is_dry(cn) and not is_heifer(cn)
            ],
            'drachtdagen' : [
                cn 
                for cn in columnnames['drachtdagen'] 
                if is_dry(cn) and not is_heifer(cn) and is_pregnant(cn)
            ],
            'aantal' : [
                cn 
                for cn in columnnames['remaining_columns']
                if is_cow(cn) and  is_dry(cn) and not '%' in cn and not 'production' in cn
            ]
        },
        'drachtig jongvee': {
            'gewicht' : [
                cn 
                for cn in columnnames['gewicht'] 
                if is_heifer(cn) and is_pregnant(cn)
            ],
            'drachtdagen' : [
                cn 
                for cn in columnnames['drachtdagen'] 
                if is_heifer(cn) and is_pregnant(cn)
            ],
            'energievoeding' : [
                cn 
                for cn in columnnames['energievoeding'] 
                if is_heifer(cn)
            ],
            'gewichtstoename' : [
                cn 
                for cn in columnnames['gewichtstoename'] 
                if is_heifer(cn)
            ],
            'aantal' : [
                cn 
                for cn in columnnames['remaining_columns']
                if is_heifer(cn) and is_pregnant(cn) and not '%' in cn and not 'production' in cn
            ]
        },
        'niet drachtig jongvee': {
            'gewicht' : [
                cn 
                for cn in columnnames['gewicht'] 
                if is_heifer(cn) and not is_pregnant(cn)
            ],
            'energievoeding' : [
                cn 
                for cn in columnnames['energievoeding'] 
                if is_heifer(cn)
            ],
            'gewichtstoename' : [
                cn 
                for cn in columnnames['gewichtstoename'] 
                if is_heifer(cn)
            ],
            'aantal' : [
                cn 
                for cn in columnnames['remaining_columns']
                if is_heifer(cn) and not is_pregnant(cn) and not '%' in cn and not 'production' in cn
            ]

        }
    }

    return categories

In [ ]:
print(json.dumps(extract_production_column_names(vera_dataframe), indent=4)) #extract_production_column_names(vera_dataframe)

{
    "melkvee": {
        "gewicht": [
            "Weight milking cows [kg]"
        ],
        "drachtdagen": [
            "Days in pregnancy (milking cows)"
        ],
        "melkproductie": [
            "Milk production [kg/animal/day]"
        ],
        "aantal": [
            "Milking cows"
        ]
    },
    "droogstaande koeien": {
        "gewicht": [
            "Weight dry cows [kg]"
        ],
        "drachtdagen": [
            "Days in pregnancy (dry cows)"
        ],
        "aantal": [
            "Dry cows"
        ]
    },
    "drachtig jongvee": {
        "gewicht": [
            "Weight heifers (pregnant) [kg]"
        ],
        "drachtdagen": [
            "Days in pregnancy (heifers)"
        ],
        "energievoeding": [
            "Energy value of feed (heifers; MJ/kg dry matter)"
        ],
        "gewichtstoename": [
            "Weight gain heifers [kg/day]"
        ],
        "aantal": [
            "Heifers (pregnant)"
        ]
    },
    "nie

In [ ]:
print(json.dumps(flatten_column_mapping(extract_production_column_names(vera_dataframe)), indent=4))

[
    "Weight milking cows [kg]",
    "Days in pregnancy (milking cows)",
    "Milk production [kg/animal/day]",
    "Milking cows",
    "Weight dry cows [kg]",
    "Days in pregnancy (dry cows)",
    "Dry cows",
    "Weight heifers (pregnant) [kg]",
    "Days in pregnancy (heifers)",
    "Energy value of feed (heifers; MJ/kg dry matter)",
    "Weight gain heifers [kg/day]",
    "Heifers (pregnant)",
    "Weight heifers (not pregnant) [kg]",
    "Energy value of feed (heifers; MJ/kg dry matter)",
    "Weight gain heifers [kg/day]",
    "Heifers (not pregnant)"
]


In [ ]:
vera_dataframe[flatten_column_mapping(extract_production_column_names(vera_dataframe))]

,Weight milking cows [kg],Days in pregnancy (milking cows),Milk production [kg/animal/day],Milking cows,Weight dry cows [kg],Days in pregnancy (dry cows),Dry cows,Weight heifers (pregnant) [kg],Days in pregnancy (heifers),Energy value of feed (heifers; MJ/kg dry matter),Weight gain heifers [kg/day],Heifers (pregnant),Weight heifers (not pregnant) [kg],Energy value of feed (heifers; MJ/kg dry matter),Weight gain heifers [kg/day],Heifers (not pregnant)
Date,,,,,,,,,,,,,,,,
2011-04-04,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-04-05,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-04-06,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-06-06,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-06-07,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-06-08,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-08-02,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-08-03,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15
2011-08-04,650,160,30.0,110,650,220,13,400,140,10,0.6,14,250,10,0.6,15



#### Emissie data

In [ ]:
#| export

def find_emission_column_names(data: DataFrame):
    '''Find column names for NH3, CO2 and temperature from the DataFrame'''

    columnnames = {
        'nh3': [col for col in data.columns if any(name in col.lower() for name in MOLECULAR_NAMES['nh3']+['nh3'])],
        'co2': [col for col in data.columns if any(name in col.lower() for name in MOLECULAR_NAMES['co2']+['co2'])],
        'temp': [col for col in data.columns if 'temperatu' in col.lower()],
        'rh': [col for col in data.columns if 'vochtigheid' in col.lower() or 'rh' in col.lower()],
        'wind': [col for col in data.columns if 'wind' in col.lower()]
    }

    return columnnames


In [ ]:
print(json.dumps(find_emission_column_names(vera_dataframe), indent=3))

{
   "nh3": [
      "NH3 inside [mg/m3]",
      "NH3 outside [mg/m3]",
      "NH3 Emission [kg/year per animal place]"
   ],
   "co2": [
      "CO2 inside [ppm]",
      "CO2 outside [ppm]"
   ],
   "temp": [
      "Outside temperature [oC]",
      "Inside temperature [oC]",
      "Total heat production corrected for temperature (hpu)"
   ],
   "rh": [
      "Outside RH [%]",
      "Inside RH [%]"
   ],
   "wind": []
}


In [ ]:
#| export

def extract_emission_column_names(
        data: DataFrame # DataFrame with measurement data
) -> dict:
    '''Extract column names for NH3, CO2 and temperature from the DataFrame'''

    columnnames = find_emission_column_names(data)

    locations = {
            'binnen': ['binnen', 'inside', 'stal'],
            'buiten': ['buiten', 'outside', 'outdoor']
        }
    
    columnmapping = {K:{} for K in locations.keys()}
    for location_key, location_items in locations.items():
        for measure, columns in columnnames.items():
            for location_name in location_items:

                columnmapping[location_key][measure] = columnmapping[location_key].get(measure,[]) + [col for col in columns if location_name in col.lower()]

    columnmapping['buiten']['wind'] = columnmapping['buiten'].get('wind', []) + columnnames.get('wind', [])
    
    return columnmapping

In [ ]:
column_mapping = extract_emission_column_names(vera_dataframe)

In [ ]:
print(json.dumps(column_mapping, indent=2)) #extract_column_names(vera_dataframe)

{
  "binnen": {
    "nh3": [
      "NH3 inside [mg/m3]"
    ],
    "co2": [
      "CO2 inside [ppm]"
    ],
    "temp": [
      "Inside temperature [oC]"
    ],
    "rh": [
      "Inside RH [%]"
    ],
    "wind": []
  },
  "buiten": {
    "nh3": [
      "NH3 outside [mg/m3]"
    ],
    "co2": [
      "CO2 outside [ppm]"
    ],
    "temp": [
      "Outside temperature [oC]"
    ],
    "rh": [
      "Outside RH [%]"
    ],
    "wind": []
  }
}


In [ ]:
[col for loc, measures in column_mapping.items() for measure, cols in measures.items() for col in cols]

['NH3 inside [mg/m3]',
 'CO2 inside [ppm]',
 'Inside temperature [oC]',
 'Inside RH [%]',
 'NH3 outside [mg/m3]',
 'CO2 outside [ppm]',
 'Outside temperature [oC]',
 'Outside RH [%]']

In [ ]:
flatten_column_mapping(column_mapping)

['NH3 inside [mg/m3]',
 'CO2 inside [ppm]',
 'Inside temperature [oC]',
 'Inside RH [%]',
 'NH3 outside [mg/m3]',
 'CO2 outside [ppm]',
 'Outside temperature [oC]',
 'Outside RH [%]']

In [ ]:
vera_dataframe[flatten_column_mapping(column_mapping)]

,NH3 inside [mg/m3],CO2 inside [ppm],Inside temperature [oC],Inside RH [%],NH3 outside [mg/m3],CO2 outside [ppm],Outside temperature [oC],Outside RH [%]
Date,,,,,,,,
2011-04-04,3.970204,1063,17.9,72.0,0,578.0,13.5,76.0
2011-04-05,3.970204,1062,17.8,71.0,0,576.0,13.5,76.0
2011-04-06,4.039857,1061,17.8,71.0,0,571.0,13.5,76.0
2011-06-06,3.970204,1060,17.8,70.0,0,569.0,13.5,76.0
2011-06-07,4.039857,1059,17.8,71.0,0,570.0,13.5,76.0
2011-06-08,3.970204,1056,NaN,NaN,0,568.0,13.4,77.0
2011-08-02,3.970204,1056,17.7,72.0,0,568.0,13.4,77.0
2011-08-03,3.970204,1056,17.7,72.0,0,568.0,13.4,77.0
2011-08-04,3.970204,1057,17.7,72.0,0,565.0,13.4,77.0


### Opschonen van data

Een randvoorwaarde van de ratiomethode is dat de concentraties van NH~3~ en het 
tracergas - in dit geval CO~2~  - op dezelfde meetpunten en met dezelfde 
meetfrequentie gemeten moeten worden. Om een goede schatting van de emissie te 
verkrijgen is het van belang dat *de concentratieratio’s per meetpunt worden 
geschat en daarna een gemiddeld van deze waarden wordt genomen*, in plaats dan 
eerst een gemiddelde concentratie van al die punten te bepalen en daarna de 
ratiomethode te gebruiken.

We verwachten dat de metingen van veschillende sensoren komen en op verschillende 
tijdstippen zijn gedaan. Om te kunnen rekenen moeten rijen volledig gevult zijn. We 
kunnen dit doen door de data te resamplen op een vast tijdsinterval (bijv. 10 minuten).



In [ ]:
#| export

def resample_data(
        data: DataFrame,    # DataFrame with measurement data
        interval: str,      # resampling interval (e.g. '10min' for 10 minutes )
        method: str         # resampling method (e.g. 'linear', 'cubic' )
    ) -> DataFrame:
    
    '''Resample data to a specified interval and interpolate missing values with the givien method'''

    return data.mask(data < 0).resample(interval).interpolate(method, limit=3).dropna(how='any')

### CO~2~ productie



In [ ]:
extract_production_column_names(vera_dataframe)

{'melkvee': {'gewicht': ['Weight milking cows [kg]'],
  'drachtdagen': ['Days in pregnancy (milking cows)'],
  'melkproductie': ['Milk production [kg/animal/day]'],
  'aantal': ['Milking cows']},
 'droogstaande koeien': {'gewicht': ['Weight dry cows [kg]'],
  'drachtdagen': ['Days in pregnancy (dry cows)'],
  'aantal': ['Dry cows']},
 'drachtig jongvee': {'gewicht': ['Weight heifers (pregnant) [kg]'],
  'drachtdagen': ['Days in pregnancy (heifers)'],
  'energievoeding': ['Energy value of feed (heifers; MJ/kg dry matter)'],
  'gewichtstoename': ['Weight gain heifers [kg/day]'],
  'aantal': ['Heifers (pregnant)']},
 'niet drachtig jongvee': {'gewicht': ['Weight heifers (not pregnant) [kg]'],
  'energievoeding': ['Energy value of feed (heifers; MJ/kg dry matter)'],
  'gewichtstoename': ['Weight gain heifers [kg/day]'],
  'aantal': ['Heifers (not pregnant)']}}

In [ ]:
for category, params in pco2_category_functions_parameters.items():
    print(f"Category: {category}")
    for param, param_info in params.items():
        print(f"  Parameter: {param}, Type: {param_info.annotation}, Default: {param_info.default}")    

Category: melkvee
  Parameter: aantal, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: melkproductie, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: drachtdagen, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: gewicht, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
Category: droogstaande koeien
  Parameter: aantal, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: melkproductie, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: drachtdagen, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: gewicht, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
Category: drachtig jongvee
  Parameter: aantal, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: energievoeding, Type: <class 'inspect._empty'>, Default: <class 'inspect._empty'>
  Parameter: drachtda

In [ ]:
default_pco2_parameters

{'melkvee': {'gewicht': 650, 'drachtdagen': 160},
 'droogstaande koeien': {'gewicht': 650,
  'drachtdagen': 220,
  'melkproductie': 0},
 'drachtig jongvee': {'gewicht': 400,
  'drachtdagen': 140,
  'energievoeding': 10.0,
  'gewichtstoename': 0.6},
 'niet drachtig jongvee': {'gewicht': 250,
  'drachtdagen': 0,
  'energievoeding': 10.0,
  'gewichtstoename': 0.6}}

In [ ]:
#| export
def calculate_pco2_production_from_data(
        data: DataFrame,
        pco2_parameters=default_pco2_parameters
) -> DataFrame:
    
    pco2_columns = extract_production_column_names(data)
    pco2_data = {}

    for category, params in pco2_category_functions_parameters.items():
        if category not in pco2_columns:
            continue

        call_parameters = {}
        for param in params.keys():
            if param not in pco2_columns[category] or len(pco2_columns[category][param]) == 0:
                call_parameters[param] = pco2_parameters[category].get(param, default_pco2_parameters[category].get(param, params[param].default))
                #print(f'Using default value {call_parameters[param]} for parameter {param} of category {category}')
                continue

            colname = pco2_columns[category][param][0]
            call_parameters[param] = data[colname]
            #print(f'Using column {colname} for parameter {param} of category {category}')

        #print(f'Call parameters for category {category}: {call_parameters}')
        pco2_data[f'PCO2_{category}'] = pco2_category_functions_mapping[category](**call_parameters)
    

        # dict(inspect.signature(PCO2_melkvee).parameters)

    return  pd.concat(pco2_data, axis=1)


In [ ]:
calculate_pco2_production_from_data(vera_dataframe)

,PCO2_melkvee,PCO2_droogstaande koeien,PCO2_drachtig jongvee,PCO2_niet drachtig jongvee
Date,,,,
2011-04-04,31.82152,2.317288,1.891889,1.380852
2011-04-05,31.82152,2.317288,1.891889,1.380852
2011-04-06,31.82152,2.317288,1.891889,1.380852
2011-06-06,31.82152,2.317288,1.891889,1.380852
2011-06-07,31.82152,2.317288,1.891889,1.380852
2011-06-08,31.82152,2.317288,1.891889,1.380852
2011-08-02,31.82152,2.317288,1.891889,1.380852
2011-08-03,31.82152,2.317288,1.891889,1.380852
2011-08-04,31.82152,2.317288,1.891889,1.380852


In [ ]:
vera_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2011-04-04 to 2012-02-16
Data columns (total 55 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Measurement institute                                  60 non-null     object 
 1   Animal Category                                        60 non-null     object 
 2   Housing system                                         60 non-null     object 
 3   Measurement location                                   60 non-null     object 
 4   Measurement period                                     60 non-null     int64  
 5   Measurement day (in period)                            60 non-null     int64  
 6   Day in year                                            60 non-null     int64  
 7   Outside temperature [oC]                               24 non-null     float64
 8   Outside RH [%]                  

### Emissie ratio

In [ ]:
#| export
def calculate_emission_ratio(
        NH3_stal,        # NH3 concentration in the barn in mg/m3
        NH3_buiten,      # NH3 concentration outside in mg/m3
        CO2_stal,        # CO2 concentration in the barn in ppm
        CO2_buiten       # CO2 concentration outside in ppm
    ):
    '''Calculate the emission ratio '''
    nh3_diff = gas_density_from_sensor_measurment(
        NH3_stal - NH3_buiten, 
        molweight=MOLECULAR_MASS['nh3'])
    
    co2_diff = gas_density_from_sensor_measurment(
        CO2_stal - CO2_buiten, 
        molweight=MOLECULAR_MASS['co2'])
    
    return nh3_diff / co2_diff  # dimensionless ratio
    

With ratios calculated we can calculate the NH~3~ emission.

### Uiteindelijke berekenening module

In [ ]:
columnmapping = extract_emission_column_names(vera_dataframe)
temperatuur = data[columnmapping['binnen']['temp']].mean(axis=1)
temperatuur

Date
2011-04-04    17.9
2011-04-05    17.8
2011-04-06    17.8
2011-06-06    17.8
2011-06-07    17.8
2011-06-08     NaN
2011-08-02    17.7
2011-08-03    17.7
2011-08-04    17.7
2011-10-06    17.7
2011-10-07    17.7
2011-10-08    17.7
2011-11-24    17.7
2011-11-25    17.8
2011-11-26    17.7
2012-01-24    17.7
2012-01-25    17.7
2012-01-26    17.7
2011-05-11    17.7
2011-07-06    17.7
2011-09-07    17.7
2011-10-26    17.7
2011-12-08    17.7
2012-02-16    17.7
2011-05-03    17.7
2011-06-27    17.8
2011-08-31    17.8
2011-11-02    17.8
2011-12-15    17.7
2012-02-14    17.6
2011-04-12    17.6
2011-06-15    17.5
2011-08-17    17.4
2011-10-12    17.3
2011-12-01    17.3
2012-01-31    17.3
2011-04-04    17.3
2011-04-05    17.2
2011-04-06    17.3
2011-06-06    17.3
2011-06-07    17.4
2011-06-08    17.5
2011-08-02    17.5
2011-08-03    17.6
2011-08-04    17.6
2011-10-06    17.7
2011-10-07    17.7
2011-10-08    17.7
2011-11-24    17.8
2011-11-25    17.8
2011-11-26    17.8
2012-01-24    17.8
2012-01

In [ ]:
pco2_calculated =     calculate_pco2_production_from_data(data)
pco2_calculated.sum(axis=1).rename('PCO2 totaal') 

Date
2011-04-04    37.411548
2011-04-05    37.411548
2011-04-06    37.411548
2011-06-06    37.411548
2011-06-07    37.411548
2011-06-08    37.411548
2011-08-02    37.411548
2011-08-03    37.411548
2011-08-04    37.411548
2011-10-06    37.411548
2011-10-07    37.411548
2011-10-08    37.411548
2011-11-24    37.411548
2011-11-25    37.411548
2011-11-26    37.411548
2012-01-24    37.411548
2012-01-25    37.411548
2012-01-26    37.411548
2011-05-11    37.411548
2011-07-06    37.411548
2011-09-07    37.411548
2011-10-26    37.411548
2011-12-08    37.411548
2012-02-16    37.411548
2011-05-03    37.411548
2011-06-27    37.411548
2011-08-31    37.411548
2011-11-02    37.411548
2011-12-15    37.411548
2012-02-14    37.411548
2011-04-12    37.411548
2011-06-15    37.411548
2011-08-17    37.411548
2011-10-12    37.411548
2011-12-01    37.411548
2012-01-31    37.411548
2011-04-04    37.411548
2011-04-05    37.411548
2011-04-06    37.411548
2011-06-06    37.411548
2011-06-07    37.411548
2011-06-08 

In [ ]:
pco2_calculated.sum(axis=1).rename('PCO2 totaal') * 5

Date
2011-04-04    187.057740
2011-04-05    187.057740
2011-04-06    187.057740
2011-06-06    187.057740
2011-06-07    187.057740
2011-06-08    187.057740
2011-08-02    187.057740
2011-08-03    187.057740
2011-08-04    187.057740
2011-10-06    187.057740
2011-10-07    187.057740
2011-10-08    187.057740
2011-11-24    187.057740
2011-11-25    187.057740
2011-11-26    187.057740
2012-01-24    187.057740
2012-01-25    187.057740
2012-01-26    187.057740
2011-05-11    187.057740
2011-07-06    187.057740
2011-09-07    187.057740
2011-10-26    187.057740
2011-12-08    187.057740
2012-02-16    187.057740
2011-05-03    187.057740
2011-06-27    187.057740
2011-08-31    187.057740
2011-11-02    187.057740
2011-12-15    187.057740
2012-02-14    187.057740
2011-04-12    187.057740
2011-06-15    187.057740
2011-08-17    187.057740
2011-10-12    187.057740
2011-12-01    187.057740
2012-01-31    187.057740
2011-04-04    187.057740
2011-04-05    187.057740
2011-04-06    187.057740
2011-06-06    187.05

In [ ]:
pco2_corrected = pd.concat([calculate_temperatuur_correctie(temperatuur).rename('temperatuur_correctie') , pco2_calculated.sum(axis=1).rename('PCO2_totaal')], axis=1)

In [ ]:
pco2_corrected

,temperatuur_correctie,PCO2_totaal
Date,,
2011-04-04,1.0084,37.411548
2011-04-05,1.0088,37.411548
2011-04-06,1.0088,37.411548
2011-06-06,1.0088,37.411548
2011-06-07,1.0088,37.411548
2011-06-08,NaN,37.411548
2011-08-02,1.0092,37.411548
2011-08-03,1.0092,37.411548
2011-08-04,1.0092,37.411548


In [ ]:
pco2_corrected['PCO2_corrected'] = pco2_corrected['PCO2_totaal'] * pco2_corrected['temperatuur_correctie']

Let op, bij berekening volgens Wagenngen is er een 
factor 0.2 in de CO~2~ productie die door de werkboeken 
pas wordt toegepat bij de debiet berekening.

In [ ]:
pco2_corrected['PCO2_corrected'] * 5

Date
2011-04-04    188.629025
2011-04-05    188.703848
2011-04-06    188.703848
2011-06-06    188.703848
2011-06-07    188.703848
2011-06-08           NaN
2011-08-02    188.778671
2011-08-03    188.778671
2011-08-04    188.778671
2011-10-06    188.778671
2011-10-07    188.778671
2011-10-08    188.778671
2011-11-24    188.778671
2011-11-25    188.703848
2011-11-26    188.778671
2012-01-24    188.778671
2012-01-25    188.778671
2012-01-26    188.778671
2011-05-11    188.778671
2011-07-06    188.778671
2011-09-07    188.778671
2011-10-26    188.778671
2011-12-08    188.778671
2012-02-16    188.778671
2011-05-03    188.778671
2011-06-27    188.703848
2011-08-31    188.703848
2011-11-02    188.703848
2011-12-15    188.778671
2012-02-14    188.853494
2011-04-12    188.853494
2011-06-15    188.928317
2011-08-17    189.003141
2011-10-12    189.077964
2011-12-01    189.077964
2012-01-31    189.077964
2011-04-04    189.077964
2011-04-05    189.152787
2011-04-06    189.077964
2011-06-06    189.07

In [ ]:
nh3_binnen = data[columnmapping['binnen']['nh3']].mean(axis=1).rename('nh3_binnen')
nh3_buiten = data[columnmapping['buiten']['nh3']].min(axis=1).rename('nh3_buiten')
co2_binnen = data[columnmapping['binnen']['co2']].mean(axis=1).rename('co2_binnen')
co2_buiten = data[columnmapping['buiten']['co2']].min(axis=1).rename('co2_buiten')


In [ ]:
nh3_binnen

Date
2011-04-04    3.970204
2011-04-05    3.970204
2011-04-06    4.039857
2011-06-06    3.970204
2011-06-07    4.039857
2011-06-08    3.970204
2011-08-02    3.970204
2011-08-03    3.970204
2011-08-04    3.970204
2011-10-06    4.039857
2011-10-07    3.970204
2011-10-08    3.970204
2011-11-24    3.970204
2011-11-25    3.970204
2011-11-26    3.970204
2012-01-24    3.970204
2012-01-25    3.970204
2012-01-26    3.970204
2011-05-11    3.970204
2011-07-06    3.970204
2011-09-07    3.970204
2011-10-26    3.970204
2011-12-08    3.970204
2012-02-16    3.970204
2011-05-03    3.970204
2011-06-27    3.970204
2011-08-31    3.970204
2011-11-02    3.970204
2011-12-15    3.970204
2012-02-14    3.970204
2011-04-12    3.830898
2011-06-15    3.761246
2011-08-17    3.691593
2011-10-12         NaN
2011-12-01         NaN
2012-01-31         NaN
2011-04-04         NaN
2011-04-05         NaN
2011-04-06         NaN
2011-06-06         NaN
2011-06-07         NaN
2011-06-08         NaN
2011-08-02         NaN
2011-0

#### We volgen even de werkboeken


ventilatie 1

# BC5 = Total_corrected_heat * 0.2 /(1e-6  * (co2_binnen - co2_buiten))

# BD5 = BC5 / (totaal_aantal_dieren)

# BE5 = BC5 8 (nh3_binnen - nh3_buiten) / 1e6 * 24*365 / (totaal_plaatsen - gesloten_plaatsen)

#### Berekening volgens Wageningen

In [ ]:

ratio = calculate_emission_ratio(
    NH3_stal=nh3_binnen,
    NH3_buiten=nh3_buiten,
    CO2_stal=co2_binnen,
    CO2_buiten=co2_buiten
).rename('ratio')


In [ ]:
emission = pd.concat([ratio, pco2_corrected['PCO2_corrected']], axis=1)

In [ ]:
emission['emission'] = (emission['ratio'] * emission['PCO2_corrected']) * 24 * 365

In [ ]:
emission

,ratio,PCO2_corrected,emission
Date,,,
2011-04-04,0.003168,37.725805,1047.076865
2011-04-05,0.003162,37.740770,1045.336883
2011-04-06,0.003190,37.740770,1054.795812
2011-06-06,0.003130,37.740770,1034.691953
2011-06-07,0.003197,37.740770,1056.952849
2011-06-08,0.003149,NaN,NaN
2011-08-02,0.003149,37.755734,1041.465523
2011-08-03,0.003149,37.755734,1041.465523
2011-08-04,0.003123,37.755734,1032.998362


In [ ]:
#| export

def calculate_emission(
        data: DataFrame,        # DataFrame with measurement data
        pco2_parameters: dict,  # parameters for the PCO2 calculation
        bezetting: dict,        # dictionary with the animal categories and their counts
        interpolate: dict=dict(interval='7min', method='linear' ), # resampling interval and method

    ):
    '''Calculate the emission using the ratio method'''

    if interpolate:
        data = resample_data(data, **interpolate)
        
    columnmapping = extract_emission_column_names(data)

    if not bezetting:
        pco2_calculated = calculate_pco2_production_from_data(data, pco2_parameters)

    else:
        # Bereken de CO2 productie -------------------------------------
        pco2_function_mapping = create_pco2_function_mapping_from_parameters(pco2_parameters)
        pco2_calculated = pd.concat(
            [
                pco2_function_mapping.get(category)(**params).rename(category)
                for category, params in bezetting.items()
            ], axis=1
        )

    temperatuur = data[columnmapping['binnen']['temp']].mean(axis=1)
    pco2_corrected = pd.concat(
        [
            calculate_temperatuur_correctie(temperatuur).rename('temperatuur_correctie') , 
            pco2_calculated.sum(axis=1).rename('PCO2_totaal')
        ], axis=1
    )
    pco2_corrected['PCO2_corrected'] = pco2_corrected['PCO2_totaal'] * pco2_corrected['temperatuur_correctie']

    nh3_binnen = data[columnmapping['binnen']['nh3']].mean(axis=1)
    nh3_buiten = data[columnmapping['buiten']['nh3']].min(axis=1)
    co2_binnen = data[columnmapping['binnen']['co2']].mean(axis=1)
    co2_buiten = data[columnmapping['buiten']['co2']].min(axis=1)

    ratio = calculate_emission_ratio(
        NH3_stal=nh3_binnen,
        NH3_buiten=nh3_buiten,
        CO2_stal=co2_binnen,
        CO2_buiten=co2_buiten
    ).rename('ratio')

    emission = pd.concat([ratio, pco2_corrected], axis=1)
    emission['emission'] = (emission['ratio'] * emission['PCO2_corrected']) * 24 * 365

    return pd.concat([emission, data], axis=1)    


In [ ]:
calculate_emission(vera_dataframe, pco2_parameters=default_pco2_parameters, bezetting={}, interpolate=dict() ).info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, 2011-04-04 to 2012-02-16
Data columns (total 60 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   ratio                                                  33 non-null     float64
 1   temperatuur_correctie                                  59 non-null     float64
 2   PCO2_totaal                                            60 non-null     float64
 3   PCO2_corrected                                         59 non-null     float64
 4   emission                                               32 non-null     float64
 5   Measurement institute                                  60 non-null     object 
 6   Animal Category                                        60 non-null     object 
 7   Housing system                                         60 non-null     object 
 8   Measurement location            

In [ ]:
extract_emission_column_names(data)

{'binnen': {'nh3': ['NH3 inside [mg/m3]'],
  'co2': ['CO2 inside [ppm]'],
  'temp': ['Inside temperature [oC]'],
  'rh': ['Inside RH [%]'],
  'wind': []},
 'buiten': {'nh3': ['NH3 outside [mg/m3]'],
  'co2': ['CO2 outside [ppm]'],
  'temp': ['Outside temperature [oC]'],
  'rh': ['Outside RH [%]'],
  'wind': []}}

### Airflow from CO2

In [ ]:
def calculate_airflow_from_co2(
        PCO2,          # CO2 production in kg per uur
        CO2_stal,     # CO2 concentration in the barn in ppm
        CO2_buiten,   # CO2 concentration outside in ppm
    ):
    '''Calculate the airflow from CO2 concentrations and production'''
    
    return PCO2 * 1e-6 *  CO2_buiten / CO2_stal  # m3 per uur

## Analyse worksheet berekeningen




### NH3 Emissie berekend [kg/dpl/jaar]

```
IF(
    ISNUMBER(BE4),
    BE4*(AO4-AP4)/1000000*24*365/(Q4-Y4),
    ""
)

```

$$
BG_i = BE_i \times \frac{ (AO_i - AP_i)}{Q_i - Y_i} \times \frac{  24 \times 365}{1000000} 
$$

* $BE_i$ is _Debiet berekend [m3/uur]_

* $AO_i$ is _NH3 concentratie stal [mg/m3]_

* $AP_i$ is _NH3 concentratie buiten [mg/m3]_

* $Y_i$ is _Afgedekte ligboxen_

* $Q_i$ is _Dierplaatsen_

### Debiet berekend [m3/uur]
```
IF(
    OR(
        BD4="",AM4="*",AM4=""
    ),
    IF(
        ISNUMBER(AQ4),
        AQ4,
        ""
    ),
    BD4*0.2/(0.000001*(AM4-AN4)))
```

$$
BE_i = BD_i \times \frac{0.2}{0.000001 \times (AM_i - AN_i)}
$$

* $BD_i$ is _Warmteproductie (totaal, gecorrigeerd door temperatuur)_

* $AM_i$ is _CO2 stal [ppm]_

* $AN_i$ is _CO2 buiten [ppm]_

* $AQ_i$ is _Debiet gemeten [m3/uur]_


From these two equations we can derive:



$$
BG_i = BD_i \times \frac{0.2}{0.000001 \times (AM_i - AN_i)} \times \frac{ (AO_i - AP_i)}{Q_i - Y_i} \times \frac{  24 \times 365}{1000000}  \Leftrightarrow
$$




$$
BG_i = BD_i \times \frac{0.2}{ (AM_i - AN_i)} \times \frac{ (AO_i - AP_i)}{Q_i - Y_i} \times   24 \times 365 \Leftrightarrow
$$


$$
BG_i =  0.2 \times BD_i \times  \frac{(AO_i - AP_i)}{ (AM_i - AN_i)} \times \frac{ 24 \times 365}{Q_i - Y_i}   
$$

$$
E_i = PCO_{2i} \cdot \frac{(NH_{3})_{i}^{stal} - (NH_{3})_{i}^{buiten} }{(CO_{2})_{i}^{stal} - CO_{2i}^{buiten} }
$$


### Warmteproductie (totaal, gecorrigeerd door temperatuur)

```
IF(
    BC4="",
    "",
    IF(
        M4="*",
        BC4,
        BC4*(1000+4*(20-M4))/1000
    )
)
```

$$
BD_i = BC_i \times \frac{1000 + 4 \times (20 - M_i)}{1000}
$$

* $BC_i$ is _Warmteproductie (totaal)_

* $M_i$ is _Temperatuur [°C]_


### Warmteproductie totaal [W]

```
=IF(SUM(AY4:BB4)=0,"",SUM(AY4:BB4))
```
$$
BC_i = \sum_{j=AY}^{BB} P_j
$$
* $BC_i$ is _Warmteproductie (totaal)_

* $P_j$ is _Warmteproductie categorie (melkvee, droogstaande koeien, drachtig jongvee, niet drachtig jongvee)_

## Warmteproductie 

### Warmteproductie categorie melkvee

```
=IF(
    OR(R4="",Z4=""),
    "",
    (5.6*(IF(AD4="",'Input voor PCO2'!$C$5,AD4))^0.75+22*Z4+1.6*0.00001*(IF(AH4="",'Input voor PCO2'!$D$5,AH4))^3)*R4/1000
)
```



$$
P_{melkvee} = \frac{5.6 (AD_i)^{0.75} + 22 Z_i + 1.6 \times 10^{-5} (AH_i)^3}{1000} \times R_i
$$

Where:

* $AD_i$ is _Gewicht melkvee [kg]_

* $AH_i$ is _Drachtdagen melkvee [dagen]_

* $Z_i$ is _Melkproductie melkvee [kg/dag]_

* $R_i$ is _Aantal melkvee_

## Calculatie vergelijking met voorbeeld data

### Standaard parameters

In [ ]:
test_parameters ={
    'melkvee': {
        'drachtdagen': 160,
        'gewicht': 650,
        'melkproductie': 28
    },
    'droogstaande koeien': {
        'drachtdagen': 220,
        'gewicht': 650,
        'melkproductie': 28
    },
    'drachtig jongvee': {
        'drachtdagen': 140,
        'gewicht': 400,
        'energievoeding': 10.0,
        'gewichtstoename': 0.6
    },
    'niet drachtig jongvee': {
        'drachtdagen': 0,
        'gewicht': 250,
        'energievoeding': 10.0,
        'gewichtstoename': 0.6
    }
}

In [ ]:
bezetting = {
    'melkvee': dict(aantal=130),
    'droogstaande koeien': dict(aantal=6),
    'drachtig jongvee': dict(aantal=0),
    'niet drachtig jongvee': dict(aantal=0)
}

### Parameters importeren

In [ ]:
test_data_filename = os.path.join(os.getcwd(), '..', 'data', 'massabalans', 'Testdata2.xlsx')
print(test_data_filename)

/home/fenke/repos/openstal/nbs/../data/massabalans/Testdata2.xlsx


In [ ]:
test_productiegegevens = pd.read_excel(test_data_filename, sheet_name='Bedrijfsproductiegegevens', header=0, index_col=0, parse_dates=True)

/tmp/ipykernel_6823/3760343135.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  test_productiegegevens = pd.read_excel(test_data_filename, sheet_name='Bedrijfsproductiegegevens', header=0, index_col=0, parse_dates=True)


In [ ]:
test_productiegegevens

,Waarde,Naam parameter in WLR rapport,Naam parameter in Slimme Stal,Hoe vaak deze waarde verandert
Parameter,,,,
Aantal dierplaatsen (=aantal ligboxen),179.000000,-,NaN,zelden
Aantal melkgevende koeien,110.000000,-,NaN,elke 3-7 dagen
Aantal droogstaande koeien,13.000000,-,NaN,elke 3-7 dagen
Aantal drachtige pinken,14.000000,-,NaN,elke 3-7 dagen
Aantal niet-drachtig jongvee,15.000000,-,NaN,elke 3-7 dagen
Melkproductie (kg/koe/dag),30.000000,Y1,NaN,elke 3 dagen
Ureumgetal (mg/100g),16.000000,-,NaN,elke 3 dagen
Mest mest besmeurd oppervlakte (m2),760.000000,-,NaN,zelden
Aantal ligboxen gesloten,21.000000,NaN,NaN,zelden


### Data importeren

In [ ]:

test_dataframe = pd.read_excel(test_data_filename, sheet_name='Ruwe Data (CARS)', header=1, index_col=0, parse_dates=True)

In [ ]:
test_dataframe

,NH3 concentratie stal (ppm),NH3 concentratie buiten (ppm),CO2 concentratie stal (ppm),CO2 concentratie buiten1 (ppm),CO2 concentratie buiten2 (ppm),Temperatuur stal1 ©,Temperatuur stal2 ©,Temperatuur buiten1 ©,Temperatuur buiten2 ©,Luchtvochtigheid stal1 (%),...,PCO2 correctie,Ventilatiedebiet (m3/u),Ventilatiedebiet (m3/dier/u),NH3 emissie (kg/u),NH3 emissie (kg/j),NH3 emissie (kg/dp/j),Ventilatiedebiet (m3/u).1,NH3 emissie (kg/u).1,NH3 emissie (kg/j).1,NH3 emissie (kg/dp/j).1
Tijd,,,,,,,,,,,,,,,,,,,,,
2025-07-04 00:00:00,5.7,0.0,1063.0,578.0,532.0,17.9,20.2,13.5,13.7,72.0,...,187.768559,77430.333781,509.410091,0.307414,2692.948507,17.043978,77776.0,0.35,3096.66,19.60
2025-07-04 00:01:00,5.7,0.0,1062.0,576.0,531.0,17.8,20.2,13.5,13.7,71.0,...,187.805971,77286.407805,508.463209,0.306843,2687.942908,17.012297,77704.0,0.36,3114.52,19.71
2025-07-04 00:02:00,5.8,0.0,1061.0,571.0,531.0,17.8,20.2,13.5,13.7,71.0,...,187.805971,76655.498353,504.312489,0.309677,2712.772456,17.169446,76969.0,0.36,3135.86,19.85
2025-07-04 00:03:00,5.7,0.0,1060.0,569.0,527.0,17.8,20.2,13.5,13.7,70.0,...,187.805971,76499.377176,503.285376,0.303718,2660.570780,16.839056,76809.0,0.35,3094.28,19.58
2025-07-04 00:04:00,5.8,0.0,1059.0,570.0,526.0,17.8,20.1,13.5,13.7,71.0,...,187.843383,76827.559310,505.444469,0.310372,2718.861545,17.207984,77228.0,0.36,3148.07,19.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-22 23:55:00,2.1,0.0,611.0,395.0,490.0,16.6,21.3,15.0,14.9,74.5,...,187.843383,173929.057883,1144.270118,0.254407,2228.607057,12.450319,183621.0,0.30,2597.60,15.28
2025-08-22 23:56:00,2.2,0.0,607.0,396.0,489.0,16.6,21.3,15.0,14.9,74.5,...,187.843383,178050.599539,1171.385523,0.272838,2390.056587,13.352271,188244.0,0.31,2691.75,15.83
2025-08-22 23:57:00,2.1,0.0,603.0,394.0,488.0,16.6,21.3,15.0,14.9,74.5,...,187.843383,179754.433028,1182.594954,0.262928,2303.249399,12.867315,189992.0,0.30,2604.49,15.32


In [ ]:
test_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 60 entries, NaT to NaT
Data columns (total 66 columns):
 #   Column                                                                                  Non-Null Count  Dtype         
---  ------                                                                                  --------------  -----         
 0   Measurement institute                                                                   60 non-null     object        
 1   Animal Category                                                                         60 non-null     object        
 2   Housing system                                                                          60 non-null     object        
 3   Measurement location                                                                    60 non-null     object        
 4   Measurement period                                                                      60 non-null     int64         
 5   Measurement day (i

In [ ]:
fmap = create_pco2_function_mapping_from_parameters(test_parameters)

In [ ]:
data = test_dataframe[['NH3 concentratie stal (ppm)',
 'NH3 concentratie buiten (ppm)',
 'CO2 concentratie stal (ppm)',
 'CO2 concentratie buiten1 (ppm)',
 'CO2 concentratie buiten2 (ppm)',
 'CO2 concentratie buiten3 (ppm)',
 'Temperatuur stal ©']].dropna().copy() #resample_data(test_dataframe, **dict(interval='7min', method='linear' ))
columnmapping = extract_column_names(data)

temperatuur = data[columnmapping['stal']['temp']].mean(axis=1)


KeyError: "None of [Index(['NH3 concentratie stal (ppm)', 'NH3 concentratie buiten (ppm)',\n       'CO2 concentratie stal (ppm)', 'CO2 concentratie buiten1 (ppm)',\n       'CO2 concentratie buiten2 (ppm)', 'CO2 concentratie buiten3 (ppm)',\n       'Temperatuur stal ©'],\n      dtype='object')] are in the [columns]"

In [ ]:
columnmapping

{'stal': {'nh3': ['NH3 concentratie stal (ppm)'],
  'co2': ['CO2 concentratie stal (ppm)'],
  'temp': ['Temperatuur stal ©']},
 'buiten': {'nh3': ['NH3 concentratie buiten (ppm)'],
  'co2': ['CO2 concentratie buiten1 (ppm)',
   'CO2 concentratie buiten2 (ppm)',
   'CO2 concentratie buiten3 (ppm)'],
  'temp': []}}

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5728 entries, 2025-08-19 00:00:00 to 2025-08-22 23:59:00
Data columns (total 7 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   NH3 concentratie stal (ppm)     5728 non-null   float64
 1   NH3 concentratie buiten (ppm)   5728 non-null   float64
 2   CO2 concentratie stal (ppm)     5728 non-null   float64
 3   CO2 concentratie buiten1 (ppm)  5728 non-null   float64
 4   CO2 concentratie buiten2 (ppm)  5728 non-null   float64
 5   CO2 concentratie buiten3 (ppm)  5728 non-null   float64
 6   Temperatuur stal ©              5728 non-null   float64
dtypes: float64(7)
memory usage: 358.0 KB


In [ ]:
bezetting

{'melkvee': {'aantal': 130},
 'droogstaande koeien': {'aantal': 6},
 'drachtig jongvee': {'aantal': 0},
 'niet drachtig jongvee': {'aantal': 0}}

In [ ]:
fmap.get('melkvee')(**bezetting['melkvee'])

np.float64(36.46325050213528)

In [ ]:
PCO2_temperatuurcorrectie(
    fmap.get('niet drachtig jongvee')(**bezetting['niet drachtig jongvee']),
    temperatuur
)

Tijd
2025-08-19 00:00:00    0.0
2025-08-19 00:01:00    0.0
2025-08-19 00:02:00    0.0
2025-08-19 00:03:00    0.0
2025-08-19 00:04:00    0.0
                      ... 
2025-08-22 23:55:00    0.0
2025-08-22 23:56:00    0.0
2025-08-22 23:57:00    0.0
2025-08-22 23:58:00    0.0
2025-08-22 23:59:00    0.0
Length: 5728, dtype: float64

In [ ]:
PCO2_temperatuurcorrectie(
    fmap.get('drachtig jongvee')(**bezetting['drachtig jongvee']),
    temperatuur
)

Tijd
2025-08-19 00:00:00    0.0
2025-08-19 00:01:00    0.0
2025-08-19 00:02:00    0.0
2025-08-19 00:03:00    0.0
2025-08-19 00:04:00    0.0
                      ... 
2025-08-22 23:55:00    0.0
2025-08-22 23:56:00    0.0
2025-08-22 23:57:00    0.0
2025-08-22 23:58:00    0.0
2025-08-22 23:59:00    0.0
Length: 5728, dtype: float64

In [ ]:
PCO2_temperatuurcorrectie(
    fmap.get('droogstaande koeien')(**bezetting['droogstaande koeien']),
    temperatuur
)

Tijd
2025-08-19 00:00:00    1.063956
2025-08-19 00:01:00    1.063528
2025-08-19 00:02:00    1.063528
2025-08-19 00:03:00    1.063956
2025-08-19 00:04:00    1.063956
                         ...   
2025-08-22 23:55:00    1.084063
2025-08-22 23:56:00    1.084063
2025-08-22 23:57:00    1.084063
2025-08-22 23:58:00    1.084491
2025-08-22 23:59:00    1.084063
Length: 5728, dtype: float64

In [ ]:
PCO2_temperatuurcorrectie(
    fmap.get('melkvee')(**bezetting['melkvee']),
    temperatuur
).rename('melkvee')

Tijd
2025-08-19 00:00:00    36.273642
2025-08-19 00:01:00    36.259056
2025-08-19 00:02:00    36.259056
2025-08-19 00:03:00    36.273642
2025-08-19 00:04:00    36.273642
                         ...    
2025-08-22 23:55:00    36.959151
2025-08-22 23:56:00    36.959151
2025-08-22 23:57:00    36.959151
2025-08-22 23:58:00    36.973736
2025-08-22 23:59:00    36.959151
Name: melkvee, Length: 5728, dtype: float64

In [ ]:
pd.concat(
        [
            PCO2_temperatuurcorrectie(
                fmap.get(category)(**params)*5,
                temperatuur
            ).rename(category)
            for category, params in bezetting.items()
        ], axis=1
    ).sum(axis=1)

Tijd
2025-08-19 00:00:00    186.687989
2025-08-19 00:01:00    186.612923
2025-08-19 00:02:00    186.612923
2025-08-19 00:03:00    186.687989
2025-08-19 00:04:00    186.687989
                          ...    
2025-08-22 23:55:00    190.216069
2025-08-22 23:56:00    190.216069
2025-08-22 23:57:00    190.216069
2025-08-22 23:58:00    190.291135
2025-08-22 23:59:00    190.216069
Length: 5728, dtype: float64

In [ ]:
emissie = calculate_emission(
    data=data,
    pco2_parameters=test_parameters,
    bezetting=bezetting,
    interpolate=dict(interval='7min', method='linear')
).resample('1h').mean()

In [ ]:
emissie / emissie.mean()

Tijd
2025-08-19 00:00:00    0.420778
2025-08-19 01:00:00    0.428174
2025-08-19 02:00:00    0.395387
2025-08-19 03:00:00    0.496850
2025-08-19 04:00:00    0.615691
                         ...   
2025-08-22 19:00:00    0.489988
2025-08-22 20:00:00    0.245229
2025-08-22 21:00:00    0.267093
2025-08-22 22:00:00    0.573332
2025-08-22 23:00:00    0.595650
Freq: h, Length: 96, dtype: float64

In [ ]:
test_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5760 entries, 2025-08-19 00:00:00 to 2025-08-22 23:59:00
Data columns (total 34 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   NH3 concentratie stal (ppm)      5758 non-null   float64
 1   NH3 concentratie buiten (ppm)    5758 non-null   float64
 2   CO2 concentratie stal (ppm)      5758 non-null   float64
 3   CO2 concentratie buiten1 (ppm)   5752 non-null   float64
 4   CO2 concentratie buiten2 (ppm)   5750 non-null   float64
 5   CO2 concentratie buiten3 (ppm)   5748 non-null   float64
 6   Temperatuur stal ©               5752 non-null   float64
 7   Temperatuur buiten1 ©            5752 non-null   float64
 8   Temperatuur buiten2 ©            5750 non-null   float64
 9   Temperatuur buiten3 ©            5748 non-null   float64
 10  Luchtvochtigheid stal (%)        5752 non-null   float64
 11  Luchtvochtigheid buiten1 (%)     5752 non-null

In [ ]:
test_dataframe

,NH3 concentratie stal (ppm),NH3 concentratie buiten (ppm),CO2 concentratie stal (ppm),CO2 concentratie buiten1 (ppm),CO2 concentratie buiten2 (ppm),CO2 concentratie buiten3 (ppm),Temperatuur stal ©,Temperatuur buiten1 ©,Temperatuur buiten2 ©,Temperatuur buiten3 ©,...,PCO2 correctie,Ventilatiedebiet (m3/u),Ventilatiedebiet (m3/dier/u),NH3 emissie (kg/u),NH3 emissie (kg/j),NH3 emissie (kg/dp/j),Ventilatiedebiet (m3/u).1,NH3 emissie (kg/u).1,NH3 emissie (kg/j).1,NH3 emissie (kg/dp/j).1
Tijd,,,,,,,,,,,,,,,,,,,,,
2025-08-19 00:00:00,2.9,0.0,896.0,369.0,491.0,452.0,21.3,19.4,19.2,20.3,...,186.687989,70849.331616,520.950968,0.143111,1253.648593,7.374403,74000.0,0.15,1324.09,7.79
2025-08-19 00:01:00,3.0,0.0,898.0,370.0,494.0,453.0,21.4,19.4,19.3,20.3,...,186.612923,70686.713361,519.755245,0.147706,1293.901172,7.611183,73763.0,0.15,1346.91,7.92
2025-08-19 00:02:00,3.0,0.0,900.0,370.0,497.0,453.0,21.4,19.4,19.2,20.3,...,186.612923,70419.971046,517.793905,0.147148,1289.018526,7.582462,73464.0,0.15,1341.45,7.89
2025-08-19 00:03:00,3.0,0.0,908.0,368.0,496.0,455.0,21.3,19.4,19.2,20.3,...,186.687989,69143.699559,508.409556,0.144481,1265.656721,7.445040,72125.0,0.15,1318.32,7.75
2025-08-19 00:04:00,3.0,0.0,910.0,367.0,494.0,456.0,21.3,19.3,19.2,20.3,...,186.687989,68761.690169,505.600663,0.143683,1258.664143,7.403907,71715.0,0.15,1312.79,7.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-08-22 23:55:00,2.1,0.0,611.0,395.0,490.0,483.0,16.6,15.0,14.9,16.4,...,190.216069,176125.989829,1295.044043,0.257621,2256.757029,13.275041,183621.0,0.30,2597.60,15.28
2025-08-22 23:56:00,2.2,0.0,607.0,396.0,489.0,484.0,16.6,15.0,14.9,16.4,...,190.216069,180299.591483,1325.732290,0.276284,2420.245859,14.236740,188244.0,0.31,2691.75,15.83
2025-08-22 23:57:00,2.1,0.0,603.0,394.0,488.0,487.0,16.6,15.0,14.9,16.5,...,190.216069,182024.946426,1338.418724,0.266249,2332.342192,13.719660,189992.0,0.30,2604.49,15.32


In [ ]:
print(json.dumps(extract_column_names(test_dataframe), indent=2))

{
  "stal": {
    "nh3": [
      "NH3 concentratie stal (ppm)",
      "NH3 concentratie stal (mg/m3)"
    ],
    "co2": [
      "CO2 concentratie stal (ppm)",
      "CO2 concentratie stal (mg/m3)"
    ],
    "temp": [
      "Temperatuur stal \u00a9"
    ]
  },
  "buiten": {
    "nh3": [
      "NH3 concentratie buiten (ppm)",
      "NH3 concentratie buiten (mg/m3)"
    ],
    "co2": [
      "CO2 concentratie buiten1 (ppm)",
      "CO2 concentratie buiten2 (ppm)",
      "CO2 concentratie buiten3 (ppm)",
      "CO2 concentatie buiten (mg/m3)"
    ],
    "temp": [
      "Temperatuur buiten1 \u00a9",
      "Temperatuur buiten2 \u00a9",
      "Temperatuur buiten3 \u00a9"
    ]
  }
}


In [ ]:
import nbdev; nbdev.nbdev_export()